<a href="https://colab.research.google.com/github/beatobongco/naive_bayesline/blob/main/NaiveBayesline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A great Naive Bayesline for classification 

Goal: understand the simple mechanics behind a really good classification baseline and implement it from scratch in Python to classify the sentiment of IMDB reviews.

![img](https://i.redd.it/w34a8bl4cfc01.jpg)

## Bayes' Theorem 

The probability of an event can be determined based on prior knowledge of conditions that might be related to the event.

### Probabilities

For a lot of things in NLP, we can think about the basis of our probabilities as "counting" things.

A toy example:

In [1]:
# the model's tiny world
tiny_corpus = "hello hello hello world"

# count the num times hello appears / num total words
# given the model's worldview, it thinks words it'll see conform to these probabilities
hello_probability = 3 / 4 # 0.75
world_probability = 1 / 4 # 0.25

# Let's simulate 
import random

n = 100000 # num simulations
splitted = tiny_corpus.split()
num_hello = 0
num_world = 0

for _ in range(n):
  c = random.choice(splitted)
  if c == "hello":
    num_hello += 1
  elif c == "world":
    num_world += 1

print(f"Percentage \"hello\" was picked {num_hello / n * 100}%. \"hello\" probability derived from counts: {hello_probability}")
print(f"Percentage \"world\" was picked {num_world / n * 100}%. \"world\" probability derived from counts: {world_probability}")

Percentage "hello" was picked 75.063%. "hello" probability derived from counts: 0.75
Percentage "world" was picked 24.937%. "world" probability derived from counts: 0.25


Naives Bayes is just like a souped up version of this!

Given that the world for our model consists solely of documents (IMDB reviews) and labels (positive or negative), and documents consist of words, we can use the counts of the words and labels in each document to derive the probabilities.

We can visualize it better if we think in terms of sets:

Each box represents a review and we can see how many reviews are positive and contain the word "happy".

We can represents this as a Venn diagram too!

![img](https://i.imgur.com/vMAaLwV.png)

### Conditional probability
What is the probability of a review being "positive" if it contains the word "happy"?

Can be expressed as 

```
P(positive | happy)
```

![img](https://imgur.com/45uAu4u.png)

And we can get that by counting the following

1. `P(positive) = no. positive reviews`
2. `P(happy) = no. reviews with word "happy"`
3. `P(positive ∩ happy) = no. positive reviews with the word happy`

![img](https://i.imgur.com/KjVAaRC.png)

We can think about evidence this way: given we have observed 4 tweets in our world containing the word "happy" (`P(happy)`), and of those tweets we know 3 of them are positive (`P(positive ∩ happy)`). 

If we trust that our evidence models real life somewhat accurately we can say

```
P(positive|happy) = P(positive ∩ happy) / P(happy)
```

For the flipped case

```
P(happy|positive) = P(happy ∩ positive) / P(positive)
```

From that we can derive Bayes Theorem:

```
# intersections are equivalent
P(positive ∩ happy) == P(happy ∩ positive) 
```

In [2]:
a = set([1, 2, 3])
b = set([2, 3, 4])

# throws an AssertionError if False!
assert a.intersection(b) == b.intersection(a)

print(f"A ∩ B = {a.intersection(b)}")
print(f"B ∩ A = {b.intersection(a)}")

A ∩ B = {2, 3}
B ∩ A = {2, 3}


```
# If
P(happy|positive) = P(happy ∩ positive) / P(positive)

# we can multiple both sides by P(positive)
P(happy|positive) * P(positive) = P(happy ∩ positive)  

# then substitute the intersection 
P(positive|happy) = P(happy|positive) * P(positive) / P(happy)
```

And that's Bayes Theorem!
```
P(A|B) = P(B|A) * P(A) / P(B)
```

## Download dataset

We can use huggingface's cool dataset library!

In [3]:
!pip install datasets

In [4]:
from datasets import load_dataset
from typing import List, Dict

In [5]:
dataset = load_dataset('imdb')

Reusing dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3)


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [7]:
dataset["train"][0]

{'label': 1,
 'text': 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!'}

In [8]:
dataset["train"][12500]

{'label': 0,
 'text': "Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly."}

## Preprocess dataset

In [9]:
import re

def preprocess(text: str) -> str:
  # lowercase
  t = text.lower()
  # remove special characters
  t = re.sub("[^\w\s]", " ", t)
  t = re.sub("\s\s+", " ", t)
  return t

def preprocess_dataset(dataset) -> List[Dict]:
  _dataset = []
  for article in dataset:
    _article = dict()
    _article["label"] = article["label"]            
    _article["text"] = preprocess(article["text"])
    _dataset.append(_article)
  return _dataset

processed_data = preprocess_dataset(dataset["train"])

In [10]:
preprocess("I am the eggman, they are the eggman...I am the walrus!Cuckoo")

'i am the eggman they are the eggman i am the walrus cuckoo'

## Build the model

### The big ideas 
1. We can determine a word's class probability by its counts 
2. The class of a document can be determined by the class probability of each of its words 
3. We will assume each word doesn't affect other words (hence: Naive)

### Python trivia: defaultdict

In [11]:
from collections import defaultdict

In [12]:
# a defaultdict specifies a default when you are accessing keys that dont yet exist
dd = defaultdict(int)
dd["wowza"]
dd["coolio"] += 1

In [13]:
dd

defaultdict(int, {'coolio': 1, 'wowza': 0})

In [14]:
dd2 = defaultdict(lambda: "laugh out loud")
dd2["lol"]

'laugh out loud'

### Ok, let's build forreals!

In [15]:
import math 

def build_model(dataset) -> Dict[str, Dict[str, float]]:
  # create a dict where 
  # word -> {positive_class_count: int, negative_class_count: int, log_probability: float}
  probs = defaultdict(lambda: defaultdict(int))
  # count the num words of each class
  totals = defaultdict(int)
  for article in dataset:
    # 0 for negative, 1 for positive
    cls = article["label"]    
    for word in article["text"].split():            
      probs[word][cls] += 1
      totals[cls] += 1

  # calculate log probabilities
  vocab_length = len(probs.keys())
  for word in probs:
    # laplace smoothing - to avoid dividing by 0
    neg_prob = probs[word][0] + 1 / (totals[0] + vocab_length)
    pos_prob = probs[word][1] + 1 / (totals[1] + vocab_length)

    # why log? too bad you didn't listen to @alron boohoo
    probs[word]["logprob"] = math.log(pos_prob / neg_prob)
  
  # not really needed for balanced classes
  # count of pos articles / count of neg articles
  prior = 1

  return probs, prior

model, prior = build_model(processed_data)

In [16]:
model["not"]

defaultdict(int, {0: 16359, 1: 14273, 'logprob': -0.13640856396196113})

In [17]:
model["good"]

defaultdict(int, {0: 7423, 1: 7724, 'logprob': 0.03974907642828515})

In [18]:
model["bad"]

defaultdict(int, {0: 7401, 1: 1907, 'logprob': -1.3560837994737982})

In [19]:
print(prior)
math.log(prior)

1


0.0

## Naive Bayes Inferencing 

We predict with the model with what's called the likelihood score

![img](https://i.imgur.com/VpNd1Ng.png)

### Log Likelihood

To prevent numerical underflow as a result of multiplying many small number together

Recall the **[First Law of ~Robotics~ Logarithms](https://www.mathcentre.ac.uk/resources/uploaded/mc-bus-loglaws-2009-1.pdf)**

```
log(A * B) = log(A) + log(B)
```

Let's kick numerical underflow's ass by using the **log** likelihood

![img](https://i.imgur.com/mLupQyi.png)

![img](https://i.imgur.com/pDTSv8N.png)

In [20]:
def predict(text, model, prior):  
  prob = math.log(prior) # 0

  # this part is called the "log likelihood"
  for word in preprocess(text).split():
    if word in model: # trivia: O(1) operation
      prob += model[word]["logprob"] 
  return 1 if prob > 0 else 0

In [21]:
predict("What a great movie! I cried tears of joy", model, prior)

1

In [22]:
predict("What a shameful movie. I cant believe I wasted my money", model, prior)

0

## Determine accuracy on the test set

In [23]:
mistakes = []
def get_accuracy(model, prior):
  global mistakes
  correct = 0
  for article in dataset["test"]:
    pred = predict(article["text"], model, prior)
    if pred == article["label"]:
      correct += 1
    else:
      mistakes.append(article)
  print(f"Accuracy is {correct / len(dataset['test']) * 100}%")  

get_accuracy(model, prior)

Accuracy is 74.732%


Let's examine our mistakes. 

Change the `seed` to get different results

In [24]:
i = 0
random.seed(1337)
random.shuffle(mistakes)
for m in mistakes:  
  t = m["text"]
  if len(t.split()) < 25:
    i += 1
    print(f"{i}. Class: {m['label']} - {t}")
    print()
    
  if i == 10:
    break

1. Class: 0 - More suspenseful, more subtle, much, much more disturbing....

2. Class: 1 - This is a great movie. Too bad it is not available on home video.

3. Class: 1 - For pure gothic vampire cheese nothing can compare to the Subspecies films. I highly recommend each and every one of them.

4. Class: 0 - Widow hires a psychopath as a handyman. Sloppy film noir thriller which doesn't make much of its tension promising set-up. (3/10)

5. Class: 1 - If you like Pauly Shore, you'll love Son in Law. If you hate Pauly Shore, then, well...I liked it!

6. Class: 1 - As a big fan of Tiny Toon Adventures, I loved this movie!!! It was so funny!!! It really captured how cartoons spent their summers.

7. Class: 1 - Very intelligent language usage of Ali, which you musn't miss! In one word: (eeh sentence...) Wicked, so keep it real and pass it on!

8. Class: 1 - This is the greatest movie ever. If you have written it off with out ever seeing it. You must give it a second try.



In [25]:
model["toon"]

defaultdict(int, {0: 5, 1: 1, 'logprob': -1.6094376586208217})

In [26]:
model["cheese"]

defaultdict(int, {0: 119, 1: 39, 'logprob': -1.1155618415402542})

In [27]:
model["suspenseful"]

defaultdict(int, {0: 69, 1: 123, 'logprob': 0.5780778486492153})

## Improve accuracy with n-grams

In [28]:
def ngram(text, n):
  word_list = text.split()
  grams = []
  for i in range(len(word_list) - n + 1):
    grams.append(tuple(word_list[i:i+n]))
  return grams


In [29]:
# bigrams
ngram("hello there that is a cute cat", 2)

[('hello', 'there'),
 ('there', 'that'),
 ('that', 'is'),
 ('is', 'a'),
 ('a', 'cute'),
 ('cute', 'cat')]

In [30]:
def build_ngram_model(dataset) -> Dict[str, Dict[str, float]]:
  """Create a dictionary where the keys are the words in the vocabulary 
  
    word -> {positive_class_count: int, 
             negative_class_count: int, 
             log_probability: float}
  """
  probs = defaultdict(lambda: defaultdict(int))
  # count the total words of each class
  totals = defaultdict(int)
  for article in dataset:
    # 0 for negative, 1 for positive
    cls = article["label"]    
    ngrams = ngram(article["text"], 2) + ngram(article["text"], 1)
    for word in ngrams:            
      probs[word][cls] += 1
      totals[cls] += 1

  # calculate log probabilities
  vocab_length = len(probs.keys())
  for word in probs:
    # laplace smoothing - to avoid dividing by 0
    neg_prob = probs[word][0] + 1 / (totals[0] + vocab_length)
    pos_prob = probs[word][1] + 1 / (totals[1] + vocab_length)
    probs[word]["logprob"] = math.log(pos_prob / neg_prob)
  
  # not really needed for balanced classes
  prior = 1

  return probs, prior

ngram_model, ngram_prior = build_ngram_model(processed_data)

In [31]:
def ngram_predict(text, model, prior):  
  prob = math.log(prior) # 0

  # this part is called the "log likelihood"
  t = preprocess(text)
  for word in ngram(t, 1) + ngram(t, 2):
    if word in model: # trivia: O(1) operation
      prob += model[word]["logprob"] 
  return 1 if prob > 0 else 0

In [32]:
ngram_predict("It was almost good but in the end it was not good", ngram_model, ngram_prior)

0

In [33]:
ngram_predict("Simply amazing. The director was sublime.", ngram_model, ngram_prior)

1

In [34]:
def get_ngram_accuracy(model, prior):
  correct = 0
  mistakes = []
  for article in dataset["test"]:
    pred = ngram_predict(article["text"], model, prior)
    if pred == article["label"]:
      correct += 1
    else:
      mistakes.append(article)
  print(f"Accuracy is {correct / len(dataset['test']) * 100}%")  

get_ngram_accuracy(ngram_model, ngram_prior)

Accuracy is 80.184%


## NBSVM

As described in the paper [Baselines and Bigrams: Simple, Good Sentiment and Topic Classification by Wang and Manning](https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf)

![img](https://i.imgflip.com/10av7r.jpg)

In [35]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(ngram_range=(1, 2))
x = vectorizer.fit_transform([d["text"] for d in processed_data])
y = np.array([d["label"] for d in processed_data])

In [36]:
# each row is a bag of words representation of a document
# (num_docs, vocab)
x.shape

(25000, 1513832)

In [37]:
def pr(x, y_i, y):
  """Count of each word belonging to class
    divided by
    How many docs in a class

    count num happy / num positive documents
  """
  # each word of y_i class's counts
  # (1, vocab_size)
  p = x[y==y_i].sum(0) 

  # counts of every positive document
  # scalar 
  class_count = (y==y_i).sum()
  # add 1 smoothing
  return (p+1) / (class_count + 1) 

In [38]:
pos_probs = pr(x, 1, y)
neg_probs = pr(x, 0, y)

# each column represents a word's positive probability
display(pos_probs)

# each column represents a word's negative probability
display(neg_probs)


print(f"Positive probabilities shape {pos_probs.shape}")
print(f"Negative probabilities shape {neg_probs.shape}")

matrix([[3.43972482e-03, 1.59987201e-04, 1.59987201e-04, ...,
         1.59987201e-04, 7.99936005e-05, 7.99936005e-05]])

matrix([[4.15966723e-03, 7.99936005e-05, 7.99936005e-05, ...,
         7.99936005e-05, 1.59987201e-04, 1.59987201e-04]])

Positive probabilities shape (1, 1513832)
Negative probabilities shape (1, 1513832)


In [39]:
# calculate log probability for each word
r = np.log(pr(x,1,y) / pr(x,0,y))
m = LogisticRegression()

# apply the log probability as a weight to each of the words
# multiplying this vector will apply each weight to each word of each review
x_nb = x.multiply(r)

# fit the model
nbsvm = m.fit(x_nb, y)

In [40]:
test_x = [t["text"] for t in preprocess_dataset(dataset["test"])]
test_x = vectorizer.transform(test_x)

# dont forget to weight the words of the test set too!
test_x = test_x.multiply(r)

In [41]:
preds = nbsvm.predict(test_x)

In [42]:
test_y = np.array([t["label"] for t in dataset["test"]])

### 90% accuracy!!

It's pretty crazy that just by weighting the features and training a logistic regression model we can get to 90%! 

In [43]:
print(f"Accuracy: {(preds == test_y).sum() / test_y.shape[0] * 100}%")

Accuracy: 90.736%


## Conclusion

### Yay
* Naive Bayes is a great baseline for classification!
* fast to train: there's no gradient descent going on...just a lot of counting

![img](https://i.imgur.com/3vkcfNz.png)

* accuracy is pretty awesome! Using BERT nets us something like +2-3% only. Here's a [leaderboard](https://paperswithcode.com/sota/sentiment-analysis-on-imdb)

### Nay
* doesn't deal with word dependencies! 
* doesn't deal with word order! 

```
I am not happy I'm dead == I am happy I'm not dead 
```

Somewhat mitigated with n-grams...

## Thank you!

So...

![img](https://i.redd.it/w34a8bl4cfc01.jpg)

> Oftentimes it's team **Naive Boy**...

## Bonus: Jeremy Howard's modification

Check out Jeremy Howard's notebook for a couple more perctage points! 

https://www.kaggle.com/jhoward/nb-svm-strong-linear-baseline

Differences
* Instead of CountVectorizer, use TFIDF weighted counts
* TFIDF hyperparams: 
  * sublinear tf
  * max_df
  * min_df
* Use higher C (less regularization) for LogisticRegression 

I *think* using these TFIDF features should generalize better

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return re_tok.sub(r' \1 ', s).split()


vectorizer = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
train_x = vectorizer.fit_transform([d["text"] for d in dataset["train"]])
train_y = np.array([d["label"] for d in dataset["train"]])

test_x = [t["text"] for t in preprocess_dataset(dataset["test"])]
test_x = vectorizer.transform(test_x)
test_y = np.array([t["label"] for t in dataset["test"]])

In [45]:
r = np.log(pr(train_x, 1, train_y) / pr(train_x, 0, train_y))

m = LogisticRegression(C=4)

x_nb = train_x.multiply(r)
nbsvm = m.fit(x_nb, train_y)

test_x = test_x.multiply(r)
preds = nbsvm.predict(test_x)

print(f"Accuracy: {(preds == test_y).sum() / test_y.shape[0] * 100}%")

Accuracy: 90.048%
